# CLTV = (Customer Value / Churn Rate) x Profit Margin

In [3]:
import pandas as pd
pd.set_option('display.max_columns', 20)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
from sklearn.preprocessing import MinMaxScaler

df_ = pd.read_excel(r"E:\4. HAFTA\online_retail_II.xlsx",
                    sheet_name="Year 2009-2010")
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


### Data Preparation

In [4]:
#selecting not include C (C's are return)
df = df[~df["Invoice"].str.contains("C", na=False)]
# greater than zero
df = df[(df['Quantity'] > 0)]

df.dropna(inplace=True)

df["TotalPrice"] = df["Quantity"] * df["Price"]
#create cltv df
cltv_df = df.groupby('Customer ID').agg({'Invoice': lambda x: len(x),
                                         'Quantity': lambda x: x.sum(),
                                         'TotalPrice': lambda x: x.sum()})
#columns name
cltv_df.columns = ['total_transaction', 'total_unit', 'total_price']

cltv_df.head()
#total trans : frequency , number of transaction
#totalunit: unit number, 
#total price: unit price x product



,total_transaction,total_unit,total_price
Customer ID,,,
12346.00000,33,70,372.86000
12347.00000,71,828,1323.32000
12348.00000,20,373,222.16000
12349.00000,102,993,2671.14000
12351.00000,21,261,300.93000


### 1. Calculate Average Order Value

In [5]:
cltv_df.shape[0]

4314

In [7]:
cltv_df['avg_order_value'] = cltv_df['total_price'] / cltv_df['total_transaction']
cltv_df['avg_order_value']

Customer ID
12346.00000   11.29879
12347.00000   18.63831
12348.00000   11.10800
12349.00000   26.18765
12351.00000   14.33000
                ...   
18283.00000    2.79030
18284.00000   16.48857
18285.00000   35.58333
18286.00000   19.34970
18287.00000   27.59659
Name: avg_order_value, Length: 4314, dtype: float64

### 2. Calculate Purchase Frequency

In [8]:
cltv_df["purchase_frequency"] = cltv_df['total_transaction'] / cltv_df.shape[0]
cltv_df["purchase_frequency"]

Customer ID
12346.00000   0.00765
12347.00000   0.01646
12348.00000   0.00464
12349.00000   0.02364
12351.00000   0.00487
                ...  
18283.00000   0.05331
18284.00000   0.00649
18285.00000   0.00278
18286.00000   0.01553
18287.00000   0.01970
Name: purchase_frequency, Length: 4314, dtype: float64

### 3. Calculate Repeat Rate and Churn Rate


In [9]:
repeat_rate = cltv_df[cltv_df.total_transaction > 1].shape[0] / cltv_df.shape[0]
churn_rate = 1 - repeat_rate

### 4. Calculate Profit Margin


In [10]:
cltv_df['profit_margin'] = cltv_df['total_price'] * 0.05
cltv_df['profit_margin']

Customer ID
12346.00000    18.64300
12347.00000    66.16600
12348.00000    11.10800
12349.00000   133.55700
12351.00000    15.04650
                 ...   
18283.00000    32.08850
18284.00000    23.08400
18285.00000    21.35000
18286.00000    64.82150
18287.00000   117.28550
Name: profit_margin, Length: 4314, dtype: float64

### 5. Calculate Customer Lifetime Value

In [11]:
cltv_df['CV'] = (cltv_df['avg_order_value'] * cltv_df["purchase_frequency"]) / churn_rate

cltv_df['CLTV'] = cltv_df['CV'] * cltv_df['profit_margin']

cltv_df.sort_values("CLTV", ascending=False)

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,CV,CLTV
Customer ID,,,,,,,,
18102.00000,627,124216,349164.35000,556.88094,0.14534,17458.21750,3754.45538,65546098.55426
14646.00000,1774,170342,248396.50000,140.02057,0.41122,12419.82500,2670.93011,33172484.52272
14156.00000,2648,108107,196566.74000,74.23215,0.61382,9828.33700,2113.62086,20773378.10442
14911.00000,5570,69722,152147.57000,27.31554,1.29115,7607.37850,1635.99538,12445636.05210
13694.00000,957,125893,131443.19000,137.34921,0.22184,6572.15950,1413.36763,9288877.52547
...,...,...,...,...,...,...,...,...
15913.00000,1,3,6.30000,6.30000,0.00023,0.31500,0.06774,0.02134
13788.00000,1,1,3.75000,3.75000,0.00023,0.18750,0.04032,0.00756
14095.00000,1,1,2.95000,2.95000,0.00023,0.14750,0.03172,0.00468


In [12]:
scaler = MinMaxScaler(feature_range=(1, 100))
scaler.fit(cltv_df[["CLTV"]])
cltv_df["SCALED_CLTV"] = scaler.transform(cltv_df[["CLTV"]])

cltv_df.sort_values("CLTV", ascending=False)


,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,CV,CLTV,SCALED_CLTV
Customer ID,,,,,,,,,
18102.00000,627,124216,349164.35000,556.88094,0.14534,17458.21750,3754.45538,65546098.55426,100.00000
14646.00000,1774,170342,248396.50000,140.02057,0.41122,12419.82500,2670.93011,33172484.52272,51.10330
14156.00000,2648,108107,196566.74000,74.23215,0.61382,9828.33700,2113.62086,20773378.10442,32.37585
14911.00000,5570,69722,152147.57000,27.31554,1.29115,7607.37850,1635.99538,12445636.05210,19.79773
13694.00000,957,125893,131443.19000,137.34921,0.22184,6572.15950,1413.36763,9288877.52547,15.02980
...,...,...,...,...,...,...,...,...,...
15913.00000,1,3,6.30000,6.30000,0.00023,0.31500,0.06774,0.02134,1.00000
13788.00000,1,1,3.75000,3.75000,0.00023,0.18750,0.04032,0.00756,1.00000
14095.00000,1,1,2.95000,2.95000,0.00023,0.14750,0.03172,0.00468,1.00000


In [13]:
cltv_df[["total_transaction", "total_unit", "total_price", "CLTV", "SCALED_CLTV"]].sort_values(by="SCALED_CLTV",
                                                                                               ascending=False).head()

,total_transaction,total_unit,total_price,CLTV,SCALED_CLTV
Customer ID,,,,,
18102.00000,627,124216,349164.35000,65546098.55426,100.00000
14646.00000,1774,170342,248396.50000,33172484.52272,51.10330
14156.00000,2648,108107,196566.74000,20773378.10442,32.37585
14911.00000,5570,69722,152147.57000,12445636.05210,19.79773
13694.00000,957,125893,131443.19000,9288877.52547,15.02980


In [14]:
cltv_df.sort_values("total_price", ascending=False)


,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,CV,CLTV,SCALED_CLTV
Customer ID,,,,,,,,,
18102.00000,627,124216,349164.35000,556.88094,0.14534,17458.21750,3754.45538,65546098.55426,100.00000
14646.00000,1774,170342,248396.50000,140.02057,0.41122,12419.82500,2670.93011,33172484.52272,51.10330
14156.00000,2648,108107,196566.74000,74.23215,0.61382,9828.33700,2113.62086,20773378.10442,32.37585
14911.00000,5570,69722,152147.57000,27.31554,1.29115,7607.37850,1635.99538,12445636.05210,19.79773
13694.00000,957,125893,131443.19000,137.34921,0.22184,6572.15950,1413.36763,9288877.52547,15.02980
...,...,...,...,...,...,...,...,...,...
15913.00000,1,3,6.30000,6.30000,0.00023,0.31500,0.06774,0.02134,1.00000
13788.00000,1,1,3.75000,3.75000,0.00023,0.18750,0.04032,0.00756,1.00000
14095.00000,1,1,2.95000,2.95000,0.00023,0.14750,0.03172,0.00468,1.00000


In [15]:
cltv_df["segment"] = pd.qcut(cltv_df["SCALED_CLTV"], 4, labels=["D", "C", "B", "A"])

cltv_df[["segment", "total_transaction", "total_unit", "total_price", "CLTV", "SCALED_CLTV"]].sort_values(
    by="SCALED_CLTV",
    ascending=False).head()


,segment,total_transaction,total_unit,total_price,CLTV,SCALED_CLTV
Customer ID,,,,,,
18102.00000,A,627,124216,349164.35000,65546098.55426,100.00000
14646.00000,A,1774,170342,248396.50000,33172484.52272,51.10330
14156.00000,A,2648,108107,196566.74000,20773378.10442,32.37585
14911.00000,A,5570,69722,152147.57000,12445636.05210,19.79773
13694.00000,A,957,125893,131443.19000,9288877.52547,15.02980


In [16]:
cltv_df.groupby("segment")[["total_transaction", "total_unit", "total_price", "CLTV", "SCALED_CLTV"]].agg(
    {"count", "mean", "sum"})

total_transaction                   total_unit                      \
                    count     sum      mean      count      sum       mean   
segment                                                                      
D                    1080   18494  17.12407       1080   117735  109.01389   
C                    1077   42239  39.21913       1077   305016  283.20891   
B                    1078   87955  81.59091       1078   733211  680.15863   
A                    1079  259007 240.04356       1079  4383262 4062.33735   

        total_price                           CLTV                  \
              count           sum       mean count             sum   
segment                                                              
D              1080  192573.08000  178.30841  1080     21819.23798   
C              1077  512708.50300  476.05246  1077    138872.87636   
B              1078 1219605.20000 1131.35918  1078    791098.68675   
A              1079 6907116.49100 6401.40546  1079 192960302.43825   

                     SCALED_CLTV                     
                mean       count        sum    mean  
segment                                              
D           20.20300        1080 1080.03296 1.00003  
C          128.94417        1077 1077.20975 1.00019  
B          733.85778        1078 1079.19487 1.00111  
A       178832.53238        1079 1370.44481 1.27011